In [1]:
import matplotlib.pyplot as plt
import fenics as df
import fenics_adjoint as dfa
import pyadjoint as pya

def backend_normalise(func):
    vec = func.vector()
    return vec.norm('l2')

In [2]:
def normalise(func, **kwargs):
    annotate = pya.annotate_tape(kwargs)

    if annotate:
        tape = dfa.get_working_tape()
        block = LogPcfBlock(func)
        tape.add_block(block)

    with pya.stop_annotating():
        output = backend_normalise(func, **kwargs)

    output = dfa.types.create_overloaded_object(output)

    if annotate:
        block.add_output(output.create_block_variable())

    return output

In [3]:
class LogPcfBlock(pya.Block):
    def __init__(self, func, **kwargs):
        super(LogPcfBlock, self).__init__()
        self.kwargs = kwargs
        self.add_dependency(func.block_variable)

    def __str__(self):
        return 'NormaliseBlock'

    def evaluate_adj(self):
        
        adj_input = self.get_outputs()[0].adj_value
        dependency = self.get_dependencies()[0]
        x = dependency.saved_output.vector()

        adj_output = x.copy()

        xnorm = x.norm('l2')

        const = 0
        for i in range(len(x)):
            const += adj_input[i][0]*x[i][0]
        const /= xnorm**3

        for i in range(len(x)):
            adj_output[i] = adj_input[i][0]/xnorm - const*x[i][0]
        dependency.add_adj_output(adj_output)

    def recompute(self):
        dependencies = self.get_dependencies()
        func = dependencies[0].saved_output
        output = backend_normalise(func)
        self.get_outputs()[0].checkpoint = output
        

In [4]:
mesh = df.UnitSquareMesh(10, 10)
V = dfa.FunctionSpace(mesh, 'CG', 1)

f = dfa.project(dfa.Expression('x[0]*x[1]', degree=1), V)
g = normalise(f)

J = dfa.assemble(g*df.dx)

h = dfa.Function(V)
h.vector()[:] = 0.1
RF = dfa.ReducedFunctional(J, dfa.Control(f))
grad = RF.derivative()
grad_np = grad.vector().get_local()
dfa.taylor_test(RF, f, h)

This integral is missing an integration domain.


UFLException: This integral is missing an integration domain.

In [ ]:
# Define Dirichlet boundary (x = 0 or x = 1)
def boundary(x):
    return x[0] < df.DOLFIN_EPS or x[0] > 1.0 - df.DOLFIN_EPS

# Define boundary condition
u0 = dfa.Constant(0.0)
bc = dfa.DirichletBC(V, u0, boundary)

# Define variational problem
u = df.TrialFunction(V)
v = df.TestFunction(V)
fe = dfa.Expression("10*exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)) / 0.02)", degree=2)
f = dfa.project(fe, V)
g = dfa.Expression("sin(5*x[0])", degree=2)
a = df.inner(df.grad(u), df.grad(v))*df.dx
L = f*v*df.dx + g*v*df.ds

# Compute solution
u = dfa.Function(V)
dfa.solve(a == L, u, bc)

In [ ]:
un = normalise(u)

In [ ]:
RF = pya.ReducedFunctional(un, pya.Control(f))
grad = RF.derivative()

In [ ]:
grad_np = grad.vector().get_local()
plt.plot(grad_np)